# Introduction
This notebook documents the Kernel Metric Network (KMN), which defines the reaction specific fingerprint (RSFP). This notebook is intended to be a standalone implementation for the ease of reproduction.

Author: Haote Li, haote.li@yale.edu

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.preprocessing import LabelEncoder
import math
from tqdm.notebook import tqdm

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.label_encoder = LabelEncoder()
        self.labels = torch.tensor(self.label_encoder.fit_transform(labels), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

     
    
class KernelMetricNetwork(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(KernelMetricNetwork, self).__init__()
        print('Using', num_classes, 'classes predictions')
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.batch_norm2 = nn.BatchNorm1d(128)

    def forward(self, x):
        x = self.dropout(self.batch_norm1(self.relu(self.fc1(x))))
        x = self.dropout(self.batch_norm2(self.relu(self.fc2(x))))
        x = self.fc3(x)
        return x


def create_data_loaders(features, labels, batch_size, train_ratio=0.9):
    dataset = CustomDataset(features, labels)
    train_size = int(train_ratio * len(dataset))
    eval_size = len(dataset) - train_size
    torch.manual_seed(2)
    train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, eval_loader

class WarmupCosineSchedule(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_epochs, total_epochs, last_epoch=-1):
        self.warmup_epochs = warmup_epochs
        self.total_epochs = total_epochs
        super(WarmupCosineSchedule, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_epochs:
            return [base_lr * (self.last_epoch / self.warmup_epochs) for base_lr in self.base_lrs]
        else:
            return [base_lr * 0.5 * (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs))) 
                    for base_lr in self.base_lrs]

def train_model(model, train_loader, eval_loader, num_epochs, lr, device, warmup_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = WarmupCosineSchedule(optimizer, warmup_epochs, num_epochs)

    best_eval_acc = 0.0
    train_loss = 0.
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        for batch_features, batch_labels in tqdm(train_loader):
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        scheduler.step()
        
        
        train_loss = train_loss/len(train_loader)
        
        model.eval()
        correct = 0
        total = 0
        eval_loss = 0.
        with torch.no_grad():
            model.eval()
            for batch_features, batch_labels in eval_loader:
                batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
                outputs = model(batch_features)
                eval_loss += criterion(outputs, batch_labels).item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += batch_labels.size(0)
                correct += (predicted == batch_labels).sum().item()
        eval_loss = eval_loss/len(eval_loader)
        eval_acc = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {eval_acc:.4f}, Train_loss {train_loss:.4f} ,Eval_Loss: {eval_loss:.4f}')
        print()
        if eval_acc > best_eval_acc:
            best_eval_acc = eval_acc
            best_model_state = model.state_dict()

    return best_model_state, best_eval_acc

def save_model(model_state, filename):
    torch.save(model_state, filename)

def load_model(model, filename):
    model.load_state_dict(torch.load(filename))
    return model

def main(features, labels, num_epochs=50, batch_size=1024, lr=0.001, warmup_epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    train_loader, eval_loader = create_data_loaders(features, labels, batch_size)
    
    input_dim = features.shape[1]
    num_classes = len(set(labels))
    model = KernelMetricNetwork(input_dim, num_classes).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {total_params}")
    
    best_model_state, best_acc = train_model(model, train_loader, eval_loader, num_epochs, lr, device, warmup_epochs)
    
    print(f"Best validation accuracy: {best_acc:.4f}")
    
    save_model(best_model_state, "best_model_50ep_4096batchsize_AdamW.pth")

In [2]:
import pickle
import numpy as np
import pandas as pd

In [3]:

features = pickle.load(open('MixFP_Reactant_Features_p4_r2_update_1024_dim.pkl', 'rb'))
# Load the premade features. This object is an int8 numpy array with dimensions [N data entries, 1024*3]



In [4]:
df = pickle.load(open('FPCompatible_Cleaned_Pistachio.pkl','rb'))
# Loading Pistachio to obtain all named reactions to create individual labels
df = df[~df['namerxndef'].isna()].reset_index(drop=True)

labels = df['namerxndef'].values 
del(df)

In [5]:
# Sanity check
valid_indices = []
for l in labels:
    if type(l) == type(''):
        valid_indices.append(True)
    else:
        valid_indices.append(False)

In [5]:
main(features, labels, num_epochs = 50, batch_size = 4096,lr=0.001)

Using 2285 classes predictions
Number of parameters: 1901549


  0%|          | 0/817 [00:00<?, ?it/s]

Epoch 1/50, Validation Accuracy: 0.0004, Train_loss 7.9725 ,Eval_Loss: 7.8573



  0%|          | 0/817 [00:00<?, ?it/s]

Epoch 2/50, Validation Accuracy: 0.8239, Train_loss 2.1127 ,Eval_Loss: 0.8009



...

Epoch 48/50, Validation Accuracy: 0.9252, Train_loss 0.2403 ,Eval_Loss: 0.2349



  0%|          | 0/817 [00:00<?, ?it/s]

Epoch 49/50, Validation Accuracy: 0.9252, Train_loss 0.2398 ,Eval_Loss: 0.2351



  0%|          | 0/817 [00:00<?, ?it/s]

Epoch 50/50, Validation Accuracy: 0.9252, Train_loss 0.2400 ,Eval_Loss: 0.2351

Best validation accuracy: 0.9252
